## Explore Toronto

**Import libraries**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

**Scrape the website for the table and assign the rows from the table**

In [124]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc = requests.get(url).text
soup = BeautifulSoup(html_doc, 'html.parser')
html_table = soup.find('table')
table_rows = html_table.find_all('tr')

**Add the rows into a dataframe**


In [125]:
l = []
for tr in table_rows:  
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
toronto = pd.DataFrame(l, columns=["PostalCode", "Borough","Neighborhood"])
toronto.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


**Remove rows with 'Not assigned' in the Borough column**  
Also remove first row and unwanted characters

In [126]:
toronto = toronto[toronto.Borough != 'Not assigned']
toronto.drop([0], axis=0, inplace=True)
toronto['Neighborhood'] = toronto['Neighborhood'].str.rstrip('\n')
toronto.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


**Group the rows by postal code and aggregate the Neighborhood values**

In [127]:
toronto = toronto.groupby(['PostalCode', 'Borough']).agg([('neighborhood', ', '.join)])
toronto.reset_index(inplace=True)
toronto.columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Replace Neighborhoods that are 'Not assigned' with the name of the borough**

In [128]:
import numpy as np
toronto['Neighborhood'] = np.where(toronto['Neighborhood'] == 'Not assigned', toronto['Borough'], toronto['Neighborhood'])
toronto.loc[85]

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 85, dtype: object

In [129]:
toronto.shape

(103, 3)